In [ ]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite fiel `\GF{p^m}`
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    
    def __init__(self, defining_set, generating_pol):
        """
        Initialize.
        """
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = generating_pol.base_ring().prime_subfield()
        
        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        self._dimension = len(defining_set) - generating_pol.degree()
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
    def get_generating_pol(self):
        return self._generating_pol
    
    def get_defining_set(self):
        return self._defining_set
    
    def parity_check_matrix(self):
        """
        Return a parity check matrix os the code.
        """
        g = self._generating_pol
        L = self._defining_set
        n = self._length
        d = g.degree()
        
        aux = vector([0 for i in range(n)])  # auxiliary vector of n columns
        H = matrix(aux)

        for i in range(0, d):
            elem = g(L[0]).inverse_of_unit() * L[0]**i   # first row
            c = vector(elem).column()
            aux = matrix(c)
            
            for j in range(1,n):
                elem = g(L[j]).inverse_of_unit() * L[j]**i
                c = vector(elem).column()
                aux = aux.augment(c)
            
            H = H.stack(aux)

        H = H.delete_rows([0])   # delete auxiliary vector
        
        return H
    
    def minimum_distance(self):
        """
        Return the minimum distance of the code.
        """
        return 2 * (self._generating_pol).degree() + 1
    
    def _repr_(self):
        """
        Representation of a Goppa code
        """
        return "[{}, {}] Goppa code".format(self.length(), self.dimension())

class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        H = self.code().parity_check_matrix()
        self._G = transpose(H).left_kernel().basis_matrix()
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self._G
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self._G

# TODO
class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._L = self.code().get_defining_set()
        
    def get_syndrome(self, word):
        """
        Return the syndrome polynomial
        """
        syndrome = 0
        
        for c in word:
            if c != 0:
                syndrome = syndrome + c*(self._generating_pol.parent().gen() - self._L[c]).inverse_mod(self._generating_pol)
            
        return syndrome
    
    def get_generating_pol(self):
        return self._generating_pol
    
    def decode_to_code(self, word):
        # TODO    
        i = 1
    
        # Paso 1
        S = self.get_syndrome(word)

        # Paso 2
        r_prev = self.get_generating_pol()
        t = self.get_generating_pol().degree()/2
        r_i = S
        U_prev = 0
        U_i = 1

        # Paso 3 y 4
        while r_prev.degree() < t or r_i.degree() >= t:
            (d, u, v) = xgcd(r_prev, r_i)
            q_i = -v/u
            r_i = d/u
            U_i = q_i * U_i + U_prev
            r_prev = r_i
            U_prev = U_i
            i += 1

        # Paso 5
        eta = (-1)^i * r_i
        sigma = U_i

        return eta, sigma

Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder
Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

In [ ]:
F = GF(2^3)
R.<x> = F[]
g = x^2 + x + 1
L = [a for a in F.list() if g(a) != 0]
C = Goppa(L, g)

E = GoppaEncoder(C)
word = vector(GF(2), (1, 0))
y = E.encode(word)

# aquí habría que modificar ``y`` añadiéndole errores

D = GoppaDecoder(C)
D.decode_to_code(y)

# **ALGORITMO DE SUGIYAMA**

1. Calcular el síndrome $S(x)$.
2. Sean $r_{-1}(x) = g(x)$, $r_0(x) = S(x)$, $U_{-1}(x) = 0$ y $U_0(x) = 1$.
3. Buscar $q_i(x)$ y $r_i(x)$ aplicando el algoritmo de Euclides para encontrar el máximo común divisor de $r_{-1}(x)$ y $r_0(x)$ para $i = 1,..., k$, hasta que $k$ cumpla que $gr(r_{k-1}(x)) \geq t$ y $gr(r_k(x)) < t$:

$$r_{i-2}(x) = r_{i-1}(x) q_i(x) + r_i(x), \qquad gr(r_i(x)) < gr(r_{i-1})(x)$$
    
4. Calcular $U_k(x)$, donde
    
$$U_i(x) = q_i(x) U_{i-1}(x) + U_{i-2}(x)$$

5. La solución viene dada por:
$$\eta(x) = (-1)^k \delta r_k(x)$$
$$\sigma(x) = \delta U_k(x)$$